In [1]:
from matplotlib.animation import FuncAnimation
from dolfin import *
parameters["form_compiler"]["representation"] = 'quadrature'
set_log_level(30)
import warnings
from ffc.quadrature.deprecation import QuadratureRepresentationDeprecationWarning
warnings.simplefilter("once", QuadratureRepresentationDeprecationWarning)
import numpy as np
from scipy import signal
from scipy.optimize import curve_fit
import matplotlib.pyplot as plt
import obspy
from obspy.imaging import spectrogram as spect
from obspy.signal.invsim import cosine_taper

import math
import time

<frozen importlib._bootstrap>:228: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


# With slow dynamics

In [ ]:
# Form compiler options
parameters["form_compiler"]["cpp_optimize"] = True
parameters["form_compiler"]["optimize"] = True

# Define mesh
lx = 1
ly = 1*lx
m_size = 50
mesh = RectangleMesh(Point(0., 0.), Point(lx, ly), m_size, int(m_size*ly/lx))
print("Plotting a RectangleMesh")
plt.figure()
plot(mesh, title="RectangleMesh")

i_VelBC = True
i_heter = False

# Material properties
mu = Constant(14e9)
lamb = Constant(14e9)
rho = Constant(2.2e3)
gamma = Constant(20)
beta_E = Constant(-2000e9) # m
# delta_E = delta_E # l
if i_heter == True:
    gamma = Expression("(x[0] < lx0) ? gamma : gamma + (500-gamma)*(x[0]-lx0)/(lx-lx0)"\
                       , degree=1, lx0=0.5*lx, lx = lx, gamma = 5)
    beta_E = Expression("(x[0] < lx0) ? gamma : gamma + (0-gamma)*(x[0]-lx0)/(lx-lx0)"\
                       , degree=1, lx0=0.1*lx, lx = lx, gamma = beta_E)
    delta_E = Expression("(x[0] < lx0) ? gamma : gamma + (0-gamma)*(x[0]-lx0)/(lx-lx0)"\
                       , degree=1, lx0=0.1*lx, lx = lx, gamma = delta_E)
tau_min = 1e-3
tau_max = 10e-3
tau = tau_min*gamma # \tau_1 in the literature

delta_t = 0.5e-6
t_total = 2e-4

t_excitation = 0.25e-4

fc = 20e3

A_amp = 1e4

g0 = 1e0

deg_dg = 1 # DG degree

def strain_from_q(q_f,i_case):
    (e11,e12,e21,e22,v1,v2,g_int) = split(q_f)
    if i_case == 0:
        strain_tensor = as_tensor([[e11,e12],\
                                   [e21,e22]])
    elif i_case == 1:
        strain_tensor = as_tensor([[e11('+'),e12('+')],\
                                   [e21('+'),e22('+')]])
    elif i_case == 2:
        strain_tensor = as_tensor([[e11('-'),e12('-')],\
                                   [e21('-'),e22('-')]])
    return strain_tensor

def mu_fun(eps):
    return mu

def stress_fun(eps):
    sig11 = lamb*(eps[0][0]+eps[1][1]) + (mu)*(eps[0][0]+eps[0][0])
    sig22 = lamb*(eps[0][0]+eps[1][1]) + (mu)*(eps[1][1]+eps[1][1])
    sig21 = (mu)*(eps[0][1]+eps[1][0])
    sig12 = (mu)*(eps[0][1]+eps[1][0])
    return as_tensor([[sig11,sig12],\
                      [sig21,sig22]])

def energy_fun(eps):
    return (lamb+2*mu)*(eps[0][0]+eps[1][1])*(eps[0][0]+eps[1][1]) \
          -2*mu*(eps[0][0]*eps[1][1] - (eps[0][1]+eps[1][0])*(eps[0][1]+eps[1][0])/4)

EPS_SMALL = 1e-3
def phig_fun(g_inti):
    return g_inti/(tau_max*(exp(-g_inti/g0*math.log(tau_max/tau_min)))\
                            *(g_inti/g0*math.log(tau_max/tau_min)* (1-ln(g_inti/g0+EPS_SMALL)) + 1) \
                    + tau_min*(g_inti)/g0*math.log(tau_max/tau_min) )

# Berjamin's model
# def phig_fun(g_inti):
#     return 1/tau_min*g_inti/(1-g_inti*g_inti)

def Max(a, b): return (a+b+abs(a-b))/Constant(2)
def Min(a, b): return (a+b-abs(a-b))/Constant(2)

# Function space
Ee11 = FiniteElement("DG", mesh.ufl_cell(), deg_dg)
Ee12 = FiniteElement("DG", mesh.ufl_cell(), deg_dg)
Ee21 = FiniteElement("DG", mesh.ufl_cell(), deg_dg)
Ee22 = FiniteElement("DG", mesh.ufl_cell(), deg_dg)
Ve1 = FiniteElement("DG", mesh.ufl_cell(), deg_dg)
Ve2 = FiniteElement("DG", mesh.ufl_cell(), deg_dg)
Ve2 = FiniteElement("DG", mesh.ufl_cell(), deg_dg)
Gint = FiniteElement("DG", mesh.ufl_cell(), deg_dg)
V_dg = FunctionSpace(mesh, MixedElement([Ee11, Ee12, Ee21, Ee22, Ve1, Ve2, Gint]))
V_out = VectorFunctionSpace(mesh,"DG", deg_dg) 

q_vout = Function(V_out)

phi, v = TrialFunction(V_dg), TestFunction(V_dg)
q_new, q_old = Function(V_dg), Function(V_dg, name='q_field')
q_rk = Function(V_dg)

q0 = Expression(("0","0","0","0","0","0","0"), degree=1, t=0, name='q0') # 
q_old = project(q0, V_dg)

v_exact = Expression("(t<tc1 ) ? A*(lx-x[0])/lx*sin(2*pi*(x[0] + c*t)) : 0 ", \
                    degree=1, t=0, tc1=t_excitation, c=fc, A=A_amp, lx = lx) # 

# v_exact2 = Expression("(t<tc1 ) ? A*(lx-x[0])/lx*sin(2*pi*(x[0] + c*t)) : 0 ", \
#                     degree=1, t=0, tc1=t_excitation, c=fc, A=A_amp2, lx = lx) # 

v_exactr = Expression(" 0 ", \
                    degree=1, t=0, tc1=t_excitation, c=fc, A=A_amp, lx = lx) # 

# f_ext = Expression(("(t<tc1 ) ? -A1*(lx-x[0])/lx*sin(2*pi*(x[0] + c*t)) : 0 ",\
#                     "(t<tc1 ) ? -A2*(lx-x[0])/lx*sin(2*pi*(x[0] + c*t)) : 0 ",\
#                     "0"), \
#                 degree=1, t=0, tc1=t_excitation, c=fc, A1=A_amp2, A2=A_amp1, lx = lx) #

DOFIN_EPS = 1e-10 # + lx/m_size/2
if deg_dg == 0: DOFIN_EPS = 1e-10 + lx/m_size/2
def left(x, on_boundary):
    return (abs(x[0]-0.) < DOFIN_EPS and abs(x[1]-ly/2) < 0.03)
def right(x, on_boundary):
    return abs(x[0]-lx) < DOFIN_EPS

boundary_markers = MeshFunction('size_t', mesh, mesh.topology().dim() - 1)

class BoundaryNR(SubDomain):
    def inside(self, x, on_boundary):
        DOFIN_EPS = 1e-10 # + lx/m_size/2
        if deg_dg == 0: DOFIN_EPS = 1e-10 + lx/m_size/2
        return on_boundary and near(x[0], lx, DOFIN_EPS)

class BoundaryNL(SubDomain):
    def inside(self, x, on_boundary):
        DOFIN_EPS = 1e-10 # + lx/m_size/2
        if deg_dg == 0: DOFIN_EPS = 1e-10 + lx/m_size/2
        return on_boundary and near(x[0], 0, DOFIN_EPS)

class BoundaryNT(SubDomain):
    def inside(self, x, on_boundary):
        DOFIN_EPS = 1e-10 # + lx/m_size/2
        if deg_dg == 0: DOFIN_EPS = 1e-10 + ly/m_size/2
        return on_boundary and near(x[1], ly, DOFIN_EPS)
    
class BoundaryNB(SubDomain):
    def inside(self, x, on_boundary):
        DOFIN_EPS = 1e-10 # + lx/m_size/2
        if deg_dg == 0: DOFIN_EPS = 1e-10 + ly/m_size/2
        return on_boundary and near(x[1], 0, DOFIN_EPS)

bx0 = BoundaryNR()
bx0.mark(boundary_markers, 0)
bx0 = BoundaryNL()
bx0.mark(boundary_markers, 1)
bx0 = BoundaryNT()
bx0.mark(boundary_markers, 0)
bx0 = BoundaryNB()
bx0.mark(boundary_markers, 1)

dss = Measure('ds', domain=mesh, subdomain_data=boundary_markers)

if i_VelBC == False:
    bc =  [DirichletBC(V_dg.sub(0), v_exact, left, method="geometric")]
else:
    bc =  [DirichletBC(V_dg.sub(4), v_exact, left, method="geometric")]
    
# Weak form
n = FacetNormal(mesh)

## First stage
(e11_1,e12_1,e21_1,e22_1,v1_1,v2_1,g_int1) = split(q_old)
strainp1 = strain_from_q(q_old,1)
stressp1 = stress_fun(strainp1)
strainm1 = strain_from_q(q_old,2)
stressm1 = stress_fun(strainm1)
stre_avg1 = ((1-g_int1('+'))*stressp1+(1-g_int1('-'))*stressm1)/2

strain1 = strain_from_q(q_old,0)
stress1 = stress_fun(strain1)
W_energy1 = energy_fun(strain1)

spec_rad1 = sqrt((lamb + 2*mu)/rho)

F = dot( (phi-q_old)/delta_t, v) * dx 

flux_innerf1 = as_vector([-v1_1, 0, -v2_1, 0, -(1-g_int1)*stress1[0][0]/rho, -(1-g_int1)*stress1[1][0]/rho, 0])
flux_innerg1 = as_vector([0, -v1_1, 0, -v2_1, -(1-g_int1)*stress1[0][1]/rho, -(1-g_int1)*stress1[1][1]/rho, 0])

F1 = F - dot(flux_innerf1, grad(v)[:,0]) * dx - dot(flux_innerg1, grad(v)[:,1]) * dx

f_ext = Expression(("(t<tc1 && sqrt((x[1]-ly/2)*(x[1]-ly/2) + (x[0]-lx/2)*(x[0]-lx/2)) < 0.03) ? 0*sin(2*pi*c*t) : 0 "\
                    ,"0","0","0"\
                    ,"(t<tc1 && sqrt((x[1]-ly/2)*(x[1]-ly/2) + (x[0]-lx/2)*(x[0]-lx/2)) < 0.03) ? A*sin(2*pi*c*t) : 0 "\
                    ,"(t<tc1 && sqrt((x[1]-ly/2)*(x[1]-ly/2) + (x[0]-lx/2)*(x[0]-lx/2)) < 0.03) ? 0*sin(2*pi*c*t) : 0 "\
                    ,"0")\
                    ,degree=1, t=0, tc1=t_excitation, c=fc, A=A_amp, lx = lx, ly=ly)

F1 += - dot(f_ext, v) * dx

r1 = as_vector([0, 0, 0, 0, 0, 0, 1/tau*W_energy1 - phig_fun(g_int1)])
F1 += - dot(r1, v) * dx

# Rusanov solver
flux1 = 1*n('+')[0]\
                *as_vector([-(v1_1('+')+v1_1('-'))/2, 0, -(v2_1('+')+v2_1('-'))/2, 0 \
                            , -stre_avg1[0,0]/rho, -stre_avg1[1,0]/rho, 0]) \
            + 1*n('+')[1]\
                *as_vector([0, -(v1_1('+')+v1_1('-'))/2, 0, -(v2_1('+')+v2_1('-'))/2 \
                            , -stre_avg1[0,1]/rho, -stre_avg1[1,1]/rho, 0]) \
            + 1.0*spec_rad1/2*jump(q_old)

F1 += dot(flux1, jump(v)) * dS + dot(flux_innerf1,v)*n[0] * dss(1) + dot(flux_innerg1,v)*n[1] * dss(1)\
                                + dot(flux_innerf1,v)*n[0] * dss(0) + dot(flux_innerg1,v)*n[1] * dss(0) 
# + dot(flux_inner1,v)*n[0] * dss(0) 

## Second stage
(e11_2,e12_2,e21_2,e22_2,v1_2,v2_2,g_int2) = split(q_rk)
strainp2 = strain_from_q(q_rk,1)
stressp2 = stress_fun(strainp2)
strainm2 = strain_from_q(q_rk,2)
stressm2 = stress_fun(strainm2)
stre_avg2 = ((1-g_int2('+'))*stressp2+(1-g_int2('-'))*stressm2)/2

strain2 = strain_from_q(q_rk,0)
stress2 = stress_fun(strain2)
W_energy2 = energy_fun(strain2)

spec_rad2 = sqrt((lamb + 2*mu)/rho)

flux_innerf2 = as_vector([-v1_2, 0, -v2_2, 0, -(1-g_int2)*stress2[0][0]/rho, -(1-g_int2)*stress2[1][0]/rho, 0])
flux_innerg2 = as_vector([0, -v1_2, 0, -v2_2, -(1-g_int2)*stress2[0][1]/rho, -(1-g_int2)*stress2[1][1]/rho, 0])

# F2 = - dot(dot(A_adv2,q_rk), grad(v)[:,0]) * dx
F2 = - dot(flux_innerf2, grad(v)[:,0]) * dx - dot(flux_innerg2, grad(v)[:,1]) * dx

# f_ext = Expression(("0","0","0","0"\
#                     ,"(t<tc1 && sqrt((x[1]-ly/2)*(x[1]-ly/2) + (x[0]-lx/2)*(x[0]-lx/2)) < 0.03) ? A*sin(2*pi*c*t) : 0 "\
#                     ,"(t<tc1 && sqrt((x[1]-ly/2)*(x[1]-ly/2) + (x[0]-lx/2)*(x[0]-lx/2)) < 0.03) ? A*sin(2*pi*c*t) : 0 "\
#                     ,"0")\
#                     ,degree=1, t=0, tc1=t_excitation, c=fc, A=A_amp, lx = lx, ly=ly)

F2 += - dot(f_ext, v) * dx

r2 = as_vector([0, 0, 0, 0, 0, 0, 1/tau*W_energy1 - phig_fun(g_int1)])

F2 += - dot(r2, v) * dx

# Rusanov solver
flux2 = 1*n('+')[0]\
                *as_vector([-(v1_2('+')+v1_2('-'))/2, 0, -(v2_2('+')+v2_2('-'))/2, 0 \
                            , -stre_avg2[0,0]/rho, -stre_avg2[1,0]/rho, 0]) \
            + 1*n('+')[1]\
                *as_vector([0, -(v1_2('+')+v1_2('-'))/2, 0, -(v2_2('+')+v2_2('-'))/2 \
                            , -stre_avg2[0,1]/rho, -stre_avg2[1,1]/rho, 0]) \
            + 1.0*spec_rad2/2*jump(q_rk)

F2 += dot(flux2, jump(v)) * dS + dot(flux_innerf2,v)*n[0] * dss(1) + dot(flux_innerg2,v)*n[1] * dss(1)\
                               + dot(flux_innerf2,v)*n[0] * dss(0) + dot(flux_innerg2,v)*n[1] * dss(0) 
F += 0.5*((F1-F) + F2)

#===========
Sta1 = time.time()
T_sol = 0
i_print = True
i_plot = False
i_file = True
#===========

xdmf_file = XDMFFile("2D-waveDG_F.xdmf")
xdmf_file.parameters["flush_output"] = True
xdmf_file.parameters["functions_share_mesh"] = True
xdmf_file.parameters["rewrite_function_mesh"] = False

t = 0
time_steps = int(t_total/delta_t)
uuu = np.zeros((time_steps,m_size,7))

a1,L1 = lhs(F1),rhs(F1)
a,L = lhs(F),rhs(F)

for n in range(time_steps):
    if i_print:
        print('===============')
        print('Time step:',n)

    for i in range(m_size):
        uuu[n,i,:] = q_old(i/(m_size-1)*lx,ly/2)[:]


    T_incre1 = time.time()
    solve(a1==L1, q_new)
    T_incre2 = time.time()
    T_sol += T_incre2-T_incre1
    q_rk.assign(q_new)


    T_incre1 = time.time()
    solve(a==L, q_new)
    T_incre2 = time.time()
    T_sol += T_incre2-T_incre1
    q_old.assign(q_new)
    if i_print:
        print('Strain:',q_old(0,ly/2)[4])

    t += delta_t
    v_exact.t = t
    f_ext.t = t
    
    if i_file: 
        v1_out = q_old.split()[4]
        v1_out.rename("Vx", "tmp")
        v2_out = q_old.split()[5]
        v2_out.rename("Vy", "tmp")
        xdmf_file.write(v1_out, n)
        xdmf_file.write(v2_out, n)
        q_vout=project(as_vector([v1_out,v2_out]))
        q_vout.rename('V','tmp')
        xdmf_file.write(q_vout, n)
        g_out = q_old.split()[6]
        g_out.rename("Damage", "tmp")
        xdmf_file.write(g_out, n)

T_total = time.time()-Sta1
if i_plot:
    plt.subplots()
    plot(q_old[2])
    plt.show()

print("Total time:",T_total)
print("Solving time:",T_sol)
# return uuu

Plotting a RectangleMesh


/home/aangniu/anaconda3/envs/fenicsproject/lib/python3.9/site-packages/ffc/jitcompiler.py:234: QuadratureRepresentationDeprecationWarning: 
*** ===================================================== ***
*** FFC: quadrature representation is deprecated! It will ***
*** likely be removed in 2018.2.0 release. Use uflacs     ***
*** representation instead.                               ***
*** ===================================================== ***
  issue_deprecation_warning()


Time step: 0
Strain: 0.0
Time step: 1
Strain: 0.0
Time step: 2
Strain: 0.0
Time step: 3
Strain: 0.0
Time step: 4
Strain: 0.0
Time step: 5
Strain: 0.0
Time step: 6
Strain: 0.0
Time step: 7
Strain: 0.0
Time step: 8
Strain: 0.0
Time step: 9
Strain: 0.0
Time step: 10
Strain: 0.0
Time step: 11
Strain: 0.0
Time step: 12
Strain: 0.0
Time step: 13
Strain: 0.0
Time step: 14
Strain: 0.0
Time step: 15
Strain: 0.0
Time step: 16
Strain: 0.0
Time step: 17
Strain: 0.0
Time step: 18
Strain: 0.0
Time step: 19
Strain: 0.0
Time step: 20
Strain: 0.0
Time step: 21
Strain: 0.0
Time step: 22
Strain: 0.0
Time step: 23
Strain: 0.0
Time step: 24
Strain: -1.12750060078e-28
Time step: 25
Strain: 4.28808628186e-28
Time step: 26
Strain: 1.11840898585e-26
Time step: 27
Strain: 4.02893053607e-26
Time step: 28
Strain: -3.66226612197e-26
Time step: 29
Strain: -6.36972801422e-25
Time step: 30
Strain: -1.71245686895e-24
Time step: 31
Strain: -1.30772972545e-25
Time step: 32
Strain: 1.0717629146e-23
Time step: 33
Strain: 

Strain: 1.99020856144e-07
Time step: 152
Strain: 3.49292065426e-07
Time step: 153
Strain: 5.09375422103e-07
Time step: 154
Strain: 6.65014938545e-07
Time step: 155
Strain: 7.98503391412e-07
Time step: 156
Strain: 8.89551790707e-07
Time step: 157
Strain: 9.16585532075e-07
Time step: 158
Strain: 8.58432174842e-07
Time step: 159
Strain: 6.96326787346e-07
Time step: 160
Strain: 4.16123024773e-07
Time step: 161
Strain: 1.05648015723e-08
Time step: 162
Strain: -5.1855230458e-07
Time step: 163
Strain: -1.1585162376e-06
Time step: 164
Strain: -1.88431971372e-06
Time step: 165
Strain: -2.65794668341e-06
Time step: 166
Strain: -3.42847910318e-06
Time step: 167
Strain: -4.13314212592e-06
Time step: 168
Strain: -4.69935760803e-06
Time step: 169
Strain: -5.04781895207e-06
Time step: 170
Strain: -5.09653794871e-06
Time step: 171
Strain: -4.76575005929e-06
Time step: 172
Strain: -3.98350225981e-06
Time step: 173
Strain: -2.69169083095e-06
Time step: 174
Strain: -8.52268691215e-07
Time step: 175
Strai